In [ ]:
import numpy as np
import cv2
#Importing these two are standard practice.

In [ ]:
cap = cv2.VideoCapture('../vid/swarm_drone/swarm_drone.mp4')
# VideoCapture puts a video into cv2.
ret,frame = cap.read()
# .read reads the object and returns True if the object has been read.
print(ret)

In [ ]:
def run(func, link='../vid/swarm_drone/swarm_drone.mp4'):
    cap = cv2.VideoCapture(link)
    while True:
         # Capture frame-by-frame
        ret, frame = cap.read()
        func(ret, frame)
        if cv2.waitKey(1) & 0xff == 27:
            break
    cap.release()
    cv2.destroyAllWindows()
# "run" will take in a function and a link.
# We will first VideoCapture the link as we did in the prev. cell.
# We want for CV to read this object (view) until we press the 27 key, 
#    which closes the window that is displaying this video/object. 

def func(ret, frame):
    cv2.imshow('frame' ,frame)
run(func)
# Very inefficeint. Basically what func does is that it displays the video using imshow. 

In [1]:
def scale(frame, factor=1):
    if factor == 1:
        return frame
    height, wid = frame.shape[:2]
    scaled_wid, scaled_height = np.int(wid * factor), np.int(height * factor)
    frame = cv2.resize(frame, (scaled_wid, scaled_height))
    return frame

# 'Scale' will rescale the video. It will take in a frame, if the frame has a factor of 1
#       nothing will happen. However, if not, then we will get the current height and width
#       of the frame by taking the first two elements of the shape list output, we will rescale
#       and reassign (resize) the frame with the new dimensions. And then return the new frame.

In [ ]:
def orb_func(ret, frame):
    frame = scale(frame)
    orb = cv2.ORB_create()
    kp = orb.detect(frame)
    # kp, des = orb.compute(frame, kp)
    kp_drawn = cv2.drawKeypoints(frame, kp, None, color=(0, 255, 0))
    cv2.imshow('kp_drawn', kp_drawn)
run(orb_func)

# (0) Idk why we even need the parameter for ret. 
# (1) Rescale the frame.
# (2) cv2.ORB_create will initiate ORB detectors. 
# (3) orb.detect(frame) will look for keypoints with ORB in our frame.
# (4) After detecting the orbs, we will draw green points for the orbs using drawKeyPoints.
# (5) We use imshow to display the current frame with the green keypoints on the image.

In [ ]:
def get_orb_kp(frame, orb=cv2.ORB_create()):
    frame = scale(frame)
    kp = orb.detect(frame)
    kp_drawn = cv2.drawKeypoints(frame, kp, None, color=(0, 255, 0))
    return kp, kp_drawn

# (1) Parameters: the frame, and initiate ORB detectors.
# (2) We will rescale the frame.
# (3) We will keypoints with ORBs in the frame.
# (4) We will color keypoints green.
# (5) This time we wil return te value returned after "detect"ing for orbs
#           in the frame and the frame with "KeyPoints."

In [ ]:
def kp_to_pairs(kp, dim):
    max_y, max_x, _ = dim
    out = np.zeros((len(kp), 2))
    for i in range(len(kp)):
        x, y = kp[i].pt
        
        out[i] = np.array([x, max_y-y])
    return out

# (1) For our paramters, we will take the list of key points (from the previous cell)
#         the key points retruend from running orb.detect(frame). And we will take dimensions. 
# (2) We will break up the dimensions from the parameter in three variables - max_y, max_x, _.
# (3) We will create a matrix of 0's with height: len(kp) and width: 2.
# (4) We will loop through the matrix and recrod the coordinate of each keypoint?
# (5) We will edit the matrix with new keypoint values?
# (6) We return a matrix with 1's where there are keypoints and 0's where there aren't.

In [ ]:
def get_centers(Z, num=10):
    # convert to np.float32
    Z = np.float32(Z)
    # define criteria and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret,label,center=cv2.kmeans(Z,num,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    return center

# (1) Parameters: Z, num = 10.
# (2) Convert Z to np.float32
# (3) We will set up our criteria, which is bascially an iteration terminator criteria. 
#      When condition is satisfied, the algorithm iteration stops. 
#      It takes in (a) type of algorithm
#                      TERM_CRITERIA_EPS (stop algorithm iteration if epsilon is reached) 
#                      TERM_CRITERIA_MAX_ITER (stop algorithm after specified number of iterations)
#                      COMBINATION of EPS and MAX_ITER (stops algorithm if EITHER is true.
#                  (b) max_iter (max number of iteration) 
#                  (c) episilon (required accuracy)
# (4) Parameters: samples of np.float32 datatype, nclusters(K) number of clusters, criteria for stopping iteration
#                 algorithm, number of attempts, type of how cluster centers should be chosen.
#     Return: density of summation of distance to center, array of label, array of centers of clusters.
# (5) We return just the array of centers of clusters.

In [ ]:
def plot_boxes(frame, name, centers, dim):
    y_max, x_max, _  = dim
    
    for i in range(len(centers)):
        x, y = centers[i]
        x, y = np.int(x), np.int(y_max - y)
        half_size = 50
        top_left = x - half_size, y - half_size
        bot_right = x + half_size, y + half_size
        frame = cv2.rectangle(frame, top_left, bot_right, (0, 255, 0),  1)
        # frame = cv2.circle(frame, (x, y), 50, (0, 255, 0),  1)
    cv2.imshow(name, frame)
    
# (1) Parameters: frame, name, array of centers, dimensions
# (2) We will split up the dimension into axes.
# (3) Loop through each center, and create a rectangle surrounding each center of some dimension.
# (4) Display the frame with the rectangles.

In [2]:
def orb_func_boxed(ret, frame):
    kp, frame = get_orb_kp(frame, orb=cv2.ORB_create(nfeatures=1000))
    dim = frame.shape
    Z = kp_to_pairs(kp, dim)
    centers = get_centers(Z, num=15)
    plot_boxes(frame, 'frame', centers, dim)
    
run(orb_func_boxed)

# (1) get_orb_kp will assign kp to an array of keypoints and frame to a frame with green keypoints.
# (2) we assign shape of frame to dim: height, width, channel
# (3) Z will be the edited matrix with keypoints? So the points with keypoints will be 1's. Else will be 0's.
# (4) We will get an array of centers from the matrix Z.
# (5) We will make boxes around the centers. 


NameError: name 'run' is not defined